In [1]:
!pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-openai chromadb bs4
!pip install -U -q transformers
!pip install -q sentence-transformers
# !pip install -q faiss-cpu
!pip install -q torch datasets
!pip install -q pypdf
!pip install -q tqdm
!pip install -q faiss-gpu
!pip install -q gradio
!pip install -q accelerate==0.21.0 \
                peft==0.4.0 \
                bitsandbytes==0.40.2 \
                trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.9/815.9 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 79.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.4/56.4 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 97.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from urllib.request import urlretrieve
import numpy as np
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain.document_loaders import PyPDFLoader, PyPDFDirectoryLoader
from langchain_community.document_loaders import BSHTMLLoader

from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import LLMChain
from langchain.schema.runnable import RunnablePassthrough
from langchain.chains import RetrievalQA
from langchain_core.output_parsers import StrOutputParser
from langchain import hub
from langchain import HuggingFaceHub
import gradio as gr
import os
from langchain.llms import HuggingFacePipeline
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
import torch
from huggingface_hub import login

import warnings
warnings.filterwarnings('ignore')

In [3]:
class PDFChatBot:
    def __init__(self):
        self.hf_token = ""
        self.prompt = None
        self.documents = None
        self.embeddings = None
        self.vectordb = None
        self.tokenizer = None
        self.model = None
        self.pipeline = None
        self.chain = None
        self.query = None
        self.directory = "/content/data/"
        self.chain = None
        self.temperature = None
        self.max_new_tokens = None
        self.repetition_penalty = None
        self.top_k = None
        self.top_p = None
        self.k_context = None

    def HF_login(self, token):
        self.hf_token = token
        login(token=self.hf_token, add_to_git_credential=True)
        return "Successfully Login into HuggingFace"


    def get_data_from_user(self, text):
        os.makedirs("data", exist_ok=True)
        files = text.split("\n")
        for url in files:
            file_path = os.path.join("data", url.rpartition("/")[2])
            urlretrieve(url, file_path)
        return "Successfully Download your Data"

    def get_file_types(self):
        file_types = set()
        for filename in os.listdir(self.directory):
            if os.path.isfile(os.path.join(self.directory, filename)):
                file_extension = filename.split(".")[-1].lower()
                file_types.add(file_extension)

        self.file_types_string = ", ".join(file_types)
        self.number_of_files = len(os.listdir(self.directory))

        return self.file_types_string, self.number_of_files

    def create_vector_store_index(self):

        file_type, _ = self.get_file_types()


        if file_type == "md":
          loader = DirectoryLoader(
              self.directory,
              glob="*.md",
              loader_cls=TextLoader,
              show_progress=True,
              )
          pages = loader.load()
          text_splitter = RecursiveCharacterTextSplitter(
          chunk_size = 512,
          chunk_overlap = 100,
          )
          self.documents = text_splitter.split_documents(pages)

        elif file_type == "pdf":
          loader = DirectoryLoader(
          self.directory,
          glob="*.pdf",
          loader_cls=PyPDFLoader,
          show_progress=True,
          )
          pages = loader.load()
          text_splitter = RecursiveCharacterTextSplitter(
          chunk_size = 512,
          chunk_overlap = 100,
          )
          self.documents = text_splitter.split_documents(pages)

        elif file_type == "csv":
          loader = DirectoryLoader(
          self.directory,
          glob="*.csv",
          loader_cls=CSVLoader,
          show_progress=True,
          )
          self.documents = loader.load()

        elif file_type == "txt":
          loader = DirectoryLoader(
          self.directory,
          glob="*.txt",
          loader_cls=TextLoader,
          show_progress=True,
          )
          pages = loader.load()
          text_splitter = RecursiveCharacterTextSplitter(
          chunk_size = 512,
          chunk_overlap = 100,
          )
          self.documents = text_splitter.split_documents(pages)

        elif file_type == "html":
          loader = DirectoryLoader(
          self.directory,
          glob="*.html",
          loader_cls=BSHTMLLoader,
          show_progress=True,
          )
          pages = loader.load()
          text_splitter = RecursiveCharacterTextSplitter(
          chunk_size = 512,
          chunk_overlap = 100,
          )
          self.documents = text_splitter.split_documents(pages)
        return "Successfully preprocess your data."

    def load_embedding_model(self):
        self.embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
        return "Successfully Download embedding model"

    def create_db(self):
        self.vectordb = FAISS.from_documents(self.documents, self.embeddings)
        return "Successfully created vector DataBase."

    def load_tokenizer_and_model(self):

        self.tokenizer = AutoTokenizer.from_pretrained('mistralai/Mistral-7B-Instruct-v0.1')
        self.tokenizer.pad_token = self.tokenizer.eos_token
        self.tokenizer.padding_side = "right"

                # bitsandbytes parameters
        # Activate 4-bit precision base model loading
        self.use_4bit = True
        # Compute dtype for 4-bit base models
        self.bnb_4bit_compute_dtype = "float16"
        # Quantization type (fp4 or nf4)
        self.bnb_4bit_quant_type = "nf4"
        # Activate nested quantization for 4-bit base models (double quantization)
        self.use_nested_quant = False

        # Set up quantization config
        self.compute_dtype = getattr(torch, self.bnb_4bit_compute_dtype)

        self.bnb_config = BitsAndBytesConfig(
                                              load_in_4bit=self.use_4bit,
                                              bnb_4bit_quant_type=self.bnb_4bit_quant_type,
                                              bnb_4bit_compute_dtype=self.compute_dtype,
                                              bnb_4bit_use_double_quant=self.use_nested_quant,
                                          )

        self.model = AutoModelForCausalLM.from_pretrained(
            'mistralai/Mistral-7B-Instruct-v0.1',
            device_map='auto',
            torch_dtype=torch.float32,
            token=True,
            quantization_config=self.bnb_config,
        )
        return "Successfully load Tokenizer and Model"

    def create_pipeline(self, temperature, max_new_tokens, repetition_penalty, top_k, top_p, k_context):
        self.temperature = temperature
        self.max_new_tokens = max_new_tokens
        self.repetition_penalty = repetition_penalty
        self.top_k = top_k
        self.top_p = top_p
        self.k_context = k_context
        pipe = pipeline(
            model=self.model,
            task='text-generation',
            tokenizer=self.tokenizer,
            temperature=temperature,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
            max_new_tokens=max_new_tokens,  # mex number of tokens to generate in the output
            repetition_penalty=repetition_penalty,  # without this output begins repeating
            top_k=top_k,
            top_p=top_p,
        )
        self.pipeline = HuggingFacePipeline(pipeline=pipe)

        self.retriever = self.vectordb.as_retriever(search_kwargs={"k": k_context}) # set here to that all parameters can be set in UI

        template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer.
        {context}
        Question: {question}
        Helpful Answer:"""
        self.prompt = PromptTemplate.from_template(template)
        return "Successfully created Pipeline and set parameters"

    def llm_chain(self):
        self.chain = LLMChain(llm=self.pipeline, prompt=self.prompt)
        return "Done"

    def inference(self, query):

        self.query = query

        if self.query is None:
            raise ValueError("Query is not set. Please provide a query.")

        self.retrieval_chain = (
          {"context": self.retriever, "question": RunnablePassthrough()}
          | self.chain
        )

        answer = self.retrieval_chain.invoke(self.query)
        context = answer.get("context")
        context_processed = [context[i].page_content.replace("\n", " ") for i in range(len(context))]
        # post_processed_answer = answer["text"].split("Helpful Answer:")[-1].strip()
        return answer.get("question"), context_processed, answer.get("text")

In [4]:
Chatbot = PDFChatBot()

In [5]:
# Chatbot.HF_login(hf_token)
# Chatbot.get_data_from_user(strings)
# Chatbot.get_file_types()
# Chatbot.create_vector_store_index()
# Chatbot.load_embedding_model()
# Chatbot.create_db()
# Chatbot.load_tokenizer_and_model()
# Chatbot.create_pipeline(temperature=0.75, max_new_tokens=512, repetition_penalty=1.1, top_k=10, top_p=0.95, k_context=5)
# Chatbot.llm_chain()

In [6]:
# query = "What is free file?"
# q, c, a = Chatbot.inference(query)

# UI

In [7]:
with gr.Blocks(gr.themes.Soft(primary_hue=gr.themes.colors.slate, secondary_hue=gr.themes.colors.purple)) as demo:

  gr.Markdown('''<h3><center>Retrieval Augmented Generation</center></h3> \n
                     RAG (Retrieval-Augmented Generation) addresses the data freshness problem in Large Language Models (LLMs) like Llama-2, which lack awareness of recent events. LLMs perceive the world only through their training data, leading to challenges when needing up-to-date information or specific datasets. To tackle this, retrieval augmentation is employed, enabling relevant external knowledge from a knowledge base to be incorporated into LLM responses.
                     RAG involves creating a knowledge base containing two types of knowledge: parametric knowledge from LLM training and source knowledge from external input. Data for the knowledge base is derived from datasets relevant to the use case, which are then processed into smaller chunks to enhance relevance and efficiency. Token embeddings, generated using models like RoBERTa, are crucial for retrieving context and meaning from the knowledge base.
                     A vector database could be used to manage and search through the embeddings efficiently. The LangChain library facilitates interactions with the knowledge base, allowing LLMs to generate responses based on retrieved information. Generative Question Answering (GQA) or Retrieval Augmented Generation (RAG) techniques instruct the LLM to craft answers using knowledge base content. To enhance trust, answers can be accompanied by citations indicating the information source.
                     RAG leverages a combination of external knowledge and LLM capabilities to provide accurate, up-to-date, and well-grounded responses. This approach is gaining traction in products such as AI search engines and conversational agents, highlighting the synergy between LLMs and robust knowledge bases.
                ''')
  with gr.Tab("Data and vectorDB Creation"):
      with gr.Row():
          with gr.Column():
              login_token= gr.components.Textbox(label = "HF token", info = "Please provide a valid token from HuggingFace. i.e : hf-*********************",type = "password")
              login_success_msg = gr.components.Textbox(label = "Success Message")

      login_btn = gr.Button("Submit Token")
      login_btn.click(Chatbot.HF_login,inputs=[login_token], outputs=[login_success_msg], show_progress=True)

      with gr.Row():
          with gr.Column():
            data_text = gr.components.Textbox(label = "URL", lines=10, info = "Provide URL seperated by new line.")
            data_success_msg = gr.components.Textbox(label = "Success Message")

      btn = gr.Button("Download Data")
      btn.click(Chatbot.get_data_from_user, inputs=[data_text], outputs=[data_success_msg], show_progress=True)

      with gr.Row():
          with gr.Column():
            data_file_types_string = gr.components.Textbox(label = "Success Message")
            data_number_of_files = gr.components.Textbox(label = "Success Message")

      btn = gr.Button("Data Details")
      btn.click(Chatbot.get_file_types, outputs=[data_file_types_string, data_number_of_files], show_progress=True)   # Chatbot.create_vector_store_index

      with gr.Row():
          with gr.Column():
            preprocess_success_msg = gr.components.Textbox(label = "Success Message")

      btn = gr.Button("Preprocess Data")
      btn.click(Chatbot.create_vector_store_index, outputs=[preprocess_success_msg], show_progress=True)

      with gr.Row():
          with gr.Column():
            embedding_model_success_msg = gr.components.Textbox(label = "Success Message")

      btn = gr.Button("Download Embedding Model")
      btn.click(Chatbot.load_embedding_model, outputs=[embedding_model_success_msg], show_progress=True)


      with gr.Row():
          with gr.Column():
            vectordb_success_msg = gr.components.Textbox(label = "Success Message")

      btn = gr.Button("Create Vector DataBase")
      btn.click(Chatbot.create_db, outputs=[vectordb_success_msg], show_progress=True)


  with gr.Tab("Model and Chain"):
      with gr.Row():
          with gr.Column():
              load_model_output= gr.components.Textbox(label = "Success Message")
      btn = gr.Button("Load Tokenizer and Model")
      btn.click(Chatbot.load_tokenizer_and_model, outputs=[load_model_output], show_progress =True)

      with gr.Row():
          with gr.Column():
              temperature = gr.Slider(label="temperature", minimum=0.1, maximum=1, value=0.1, step=0.05)
              max_new_tokens = gr.Slider(label="max_new_tokens", minimum=1, maximum=2048, value=512, step=1)
              repetition_penalty = gr.Slider(label="repetition_penalty", minimum=0, maximum=2, value=1.1, step=0.1)
              top_k= gr.Slider(label="top_k", minimum=1, maximum=1000, value=10, step=1)
              top_p=gr.Slider(label="top_p", minimum=0, maximum=1, value=0.95, step=0.05)
              k_context=gr.Slider(label="k_context", minimum=1, maximum=15, value=5, step=1)
              create_model_pipeline= gr.components.Textbox(label = "Success Message")
      btn = gr.Button("Create Pipeline")
      btn.click(Chatbot.create_pipeline, inputs=[temperature, max_new_tokens, repetition_penalty, top_k, top_p, k_context], outputs=[create_model_pipeline], show_progress =True)


      with gr.Row():
          with gr.Column():
              llm_success_msg= gr.components.Textbox(label = "Success Message")
      btn = gr.Button("Create LLM Chain")
      btn.click(Chatbot.llm_chain, outputs=[llm_success_msg], show_progress =True)




  with gr.Tab("Inference"):
      with gr.Row():
          with gr.Column():
              inference_input = gr.components.Textbox(label = "Submit", info = "Write query related to your docs.")
              inference_output_question = gr.components.Textbox(label = "Question")
              inference_output_context = gr.components.Textbox(label = "Relevant Text", lines=10)
              inference_output_text = gr.components.Textbox(label = "Answer")

      btn = gr.Button("Generate")
      btn.click(Chatbot.inference, inputs=[inference_input], outputs=[inference_output_question, inference_output_context, inference_output_text], show_progress=True)



if __name__ == "__main__":
    demo.launch(share=True, debug=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://81a02cc0db1314649a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Token is valid (permission: read).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


100%|██████████| 1/1 [00:01<00:00,  1.00s/it]


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://81a02cc0db1314649a.gradio.live
